### 现在我们基本了解了bedrock的sdk, knowledge base和langchain集成的接口。现在我们进行一个实践项目

## 前言

如今很多交易所要求上市公司出了财报以外，还需提交ESG报告，披露公司在环境、社会和公司治理等方面信息，ESG报告的撰写是高度专业性工作，内容广泛，要求复杂。而生成式AI的发展，很大程度上可以简化ESG报告的撰写。利用 RAG 技术，结合企业私有ESG等数据，基于 LLM 和智能搜索，以低代码/无代码的形式，简单快速地构建企业私域知识库，配合提示词工程，可为企业提供高质量的写作建议，甚至生成报告。接下来我们让我们一起体验Bedrock Knowledge Base和Claude3如何帮助公司生成ESG报告。

## Bedrock Knowledge Base 检索ESG报告相关内容

In [ ]:
import boto3
from botocore.auth import SigV4Auth
from botocore.awsrequest import AWSRequest
from requests_aws4auth import AWS4Auth
import botocore.session
from langchain.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models import BedrockChat

question = 'ZTO Express Water Management'

# 1. set up bedrock knowledge base client and retrieve
kb_client = boto3.client('bedrock-agent-runtime')
kb_response = kb_client.retrieve(
    knowledgeBaseId='your-bedrock-konwledgebase-id',
    retrievalQuery={
        'text': question
    },
    retrievalConfiguration={
        'vectorSearchConfiguration': {
            'numberOfResults': 10,
            'overrideSearchType': 'HYBRID'
        }
    }
    #nextToken='string'
)
context = ""
for result in kb_response['retrievalResults']:
    context = context +'\n'+ result['content']['text']
    print(result['location'])
    print(result['content']['text'])
    print("----------------")


## Claude3 基于检索到的文本生成回答

In [ ]:
# 2. set up llm - claude3
claude3_kwargs = {
    "temperature":0.0001,
    #"temperature":1,
    "top_k":250,
    "top_p":0.999,
    "max_tokens": 3000,
}
claude3 = BedrockChat(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    model_kwargs=claude3_kwargs
)
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:"""
prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_01 = LLMChain(llm=claude3, prompt=prompt)

response_01=chain_01.invoke({'question': question,'context': context})
print(response_01['text'])

## Claude3 针对某个ESG议题生成报告

In [ ]:
prompt_template_02 = """
你是撰写ESG（环境、社会和治理）报告的专家。

本次,我们需要您为一家{industry}行业的公司,就 GRI 重要性议题"{topic}"编写 ESG 报告内容。

公司提供的相关数据:
{statistics}

此外,您将参考其他公司的 ESG 信息:
{relevant_info}

报告内容要求:
1. 字数在 5000 字以上
2. 遵循 GRI 标准框架的要求
3. 阐述公司在该议题下的管理政策、具体行动及未来规划
4. 根据数据给出一些详细的实际案例
5. 报告结构合理,行文条理清晰,信息连贯一致

请按照上述要求,为我们撰写高质量的 ESG 报告内容。
"""

prompt_02 = PromptTemplate(
    template=prompt_template_02, input_variables=["industry","topic","statistics","relevant_info"]
)

claude3_kwargs = {
    #"temperature":0.0001,
    "temperature":1,
    "top_k":250,
    "top_p":1,
    "max_tokens": 3000,
}
claude3 = BedrockChat(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    model_kwargs=claude3_kwargs
)
chain_02 = LLMChain(llm=claude3, prompt=prompt_02, verbose=True)

result = chain_02.invoke({
    "industry": "物流",
    "topic": "水资源和污水",
    "statistics":'''2022年废水排放量为 2,00.00 万 m3，废水排放强度为 2.22m3/ 万元，中水回用量为 66.66 万 m3。''',
    "relevant_info": response_01['text']
})

print(result['text'])